In [3]:
import xml.etree.ElementTree as ET
import glob
import os
import json


def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


classes = []
input_dir = "test"
output_dir = "labels_test"
image_dir = "test"

# create the labels folder (output directory)
os.mkdir(output_dir)

# identify all the xml files in the annotations folder (input directory)
files = glob.glob(os.path.join(input_dir, '*.xml'))
# loop through each 
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file
#     if not os.path.exists(os.path.join(image_dir, f"{filename}.png" or f"{filename}.jpg")):
#         print(f"{filename} image does not exist!")
#         continue

    result = []

    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for 
        new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))


In [96]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

In [112]:
import pandas as pd
a=pd.read_csv(r'C:\Users\VyshakB\Desktop\fasterrcn\models-master\models\research\object_detection\images\test_labels.csv')
a.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,10cabinet.PNG,1360,617,cabinet,548,431,682,556
1,11cabinet.PNG,1359,611,cabinet,237,346,432,512
2,12cabinet.PNG,1364,616,cabinet,485,302,601,379
3,1bollard.PNG,1365,613,bollard,888,262,916,349
4,1bollard.PNG,1365,613,bollard,1026,266,1065,361


In [123]:
for i in a['class'].unique():
    a['class'][a['class']==i]=list(classes.values()).index(i)

C:\Users\VyshakB\.conda\envs\torch\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [174]:
for i in range(len(a)):
    bbox=xml_to_yolo_bbox(list(a[i:i+1][['xmin','ymin','xmax','ymax']].values[0]),a[i:i+1]['width'].values[0],a[i:i+1]['height'].values[0])
    print(bbox)
    a['xx'][i:i+1]=bbox[0]
    a['yy'][i:i+1]=bbox[1]
    a['xy'][i:i+1]=bbox[2]
    a['yx'][i:i+1]=bbox[3]

[0.4522058823529412, 0.799837925445705, 0.09852941176470588, 0.2025931928687196]
[0.24613686534216336, 0.7021276595744681, 0.1434878587196468, 0.27168576104746317]
[0.3980938416422287, 0.5527597402597403, 0.08504398826979472, 0.125]
[0.6608058608058608, 0.49836867862969003, 0.020512820512820513, 0.14192495921696574]
[0.765934065934066, 0.5114192495921697, 0.02857142857142857, 0.15497553017944535]
[0.25656934306569346, 0.6405492730210016, 0.14087591240875913, 0.19224555735056542]
[0.32234432234432236, 0.6846774193548387, 0.14945054945054945, 0.2661290322580645]
[0.5897435897435898, 0.6644951140065146, 0.05274725274725275, 0.09120521172638436]
[0.6700879765395894, 0.44788273615635177, 0.10410557184750734, 0.18241042345276873]
[0.5878279883381924, 0.4688995215311005, 0.06341107871720117, 0.17862838915470494]
[0.5896501457725948, 0.4625199362041467, 0.061224489795918366, 0.15629984051036683]
[0.32536764705882354, 0.7009803921568627, 0.08308823529411764, 0.17320261437908496]
[0.568397951719

C:\Users\VyshakB\.conda\envs\torch\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\VyshakB\.conda\envs\torch\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\VyshakB\.conda\envs\torch\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\VyshakB\.conda\

In [175]:
a

,filename,width,height,class,xmin,ymin,xmax,ymax,xx,yy,xy,yx
0,10cabinet.PNG,1360,617,5,548,431,682,556,0.452206,0.799838,0.098529,0.202593
1,11cabinet.PNG,1359,611,5,237,346,432,512,0.246137,0.702128,0.143488,0.271686
2,12cabinet.PNG,1364,616,5,485,302,601,379,0.398094,0.552760,0.085044,0.125000
3,1bollard.PNG,1365,613,1,888,262,916,349,0.660806,0.498369,0.020513,0.141925
4,1bollard.PNG,1365,613,1,1026,266,1065,361,0.765934,0.511419,0.028571,0.154976
...,...,...,...,...,...,...,...,...,...,...,...,...
74,traffic_light38.png,1366,768,8,570,262,610,342,0.431918,0.393229,0.029283,0.104167
75,traffic_light40.png,1366,768,8,249,288,299,398,0.200586,0.446615,0.036603,0.143229
76,traffic_light40.png,1366,768,8,569,292,619,402,0.434846,0.451823,0.036603,0.143229
77,traffic_light40.png,1366,768,8,808,309,858,419,0.609810,0.473958,0.036603,0.143229


In [176]:
a=a[['filename','class','xx','yy','xy','yx']]

In [140]:
list(a[1:2][['xmin','xmax']].values[0])

[237, 432]

In [50]:
from sklearn.preprocessing import LabelEncoder

In [51]:
le=LabelEncoder()

In [53]:
a['class']=le.fit_transform(a['class'])

In [62]:
a

,filename,class,xx,yy,xy,yx
0,100cabinet.PNG,5,0.804156,0.680978,0.035894,0.085205
1,101cabinet.PNG,5,0.646671,0.655963,0.039259,0.077982
2,10cabinet.PNG,5,0.452206,0.799838,0.049265,0.101297
3,11cabinet.PNG,5,0.246137,0.702128,0.071744,0.135843
4,12cabinet.PNG,5,0.398094,0.552760,0.042522,0.062500
...,...,...,...,...,...,...
2409,traffic_light96.png,8,0.661054,0.484375,0.022694,0.078125
2410,traffic_light97.png,8,0.326501,0.445964,0.011713,0.044922
2411,traffic_light97.png,8,0.757687,0.511719,0.016105,0.040365
2412,traffic_light98.png,8,0.566618,0.486979,0.016105,0.040365


In [108]:
classes={0:'barrier',1:'bollard',2:'busstop',3:'busstop_road',4:'busstop_sign',
       5:'cabinet',6: 'dyellow',7: 'postbox',8: 'traffic'}

In [185]:
print(list(classes.values()))

['barrier', 'bollard', 'busstop', 'busstop_road', 'busstop_sign', 'cabinet', 'dyellow', 'postbox', 'traffic']


In [180]:

gb=a.groupby('filename')
gb.first()

,class,xx,yy,xy,yx
filename,,,,,
10cabinet.PNG,5,0.452206,0.799838,0.098529,0.202593
11cabinet.PNG,5,0.246137,0.702128,0.143488,0.271686
12cabinet.PNG,5,0.398094,0.552760,0.085044,0.125000
1bollard.PNG,1,0.660806,0.498369,0.020513,0.141925
1cabinet.PNG,5,0.256569,0.640549,0.140876,0.192246
2cabinet.PNG,5,0.322344,0.684677,0.149451,0.266129
3cabinet.PNG,5,0.589744,0.664495,0.052747,0.091205
4cabinet.PNG,5,0.670088,0.447883,0.104106,0.182410
5cabinet.PNG,5,0.587828,0.468900,0.063411,0.178628


In [182]:
for i in a['filename'].unique():
    h=gb.get_group(i)
    print(h)
#     print(i.split('.')[0])
    h.to_csv(f'test_txt/{i.split(".")[0]}.txt',index=False,header=False,sep=' ')

  class        xx        yy        xy        yx
0     5  0.452206  0.799838  0.098529  0.202593
  class        xx        yy        xy        yx
1     5  0.246137  0.702128  0.143488  0.271686
  class        xx       yy        xy     yx
2     5  0.398094  0.55276  0.085044  0.125
  class        xx        yy        xy        yx
3     1  0.660806  0.498369  0.020513  0.141925
4     1  0.765934  0.511419  0.028571  0.154976
  class        xx        yy        xy        yx
5     5  0.256569  0.640549  0.140876  0.192246
  class        xx        yy        xy        yx
6     5  0.322344  0.684677  0.149451  0.266129
  class        xx        yy        xy        yx
7     5  0.589744  0.664495  0.052747  0.091205
  class        xx        yy        xy       yx
8     5  0.670088  0.447883  0.104106  0.18241
  class        xx      yy        xy        yx
9     5  0.587828  0.4689  0.063411  0.178628
   class       xx       yy        xy      yx
10     5  0.58965  0.46252  0.061224  0.1563
   class    

In [167]:
a

,filename,width,height,class,xmin,ymin,xmax,ymax,xx,yy,xy,yx
0,10cabinet.PNG,1360,617,5,548,431,682,556,0.054539,0.438151,0.035871,0.141927
1,11cabinet.PNG,1359,611,5,237,346,432,512,0.054539,0.438151,0.035871,0.141927
2,12cabinet.PNG,1364,616,5,485,302,601,379,0.054539,0.438151,0.035871,0.141927
3,1bollard.PNG,1365,613,1,888,262,916,349,0.054539,0.438151,0.035871,0.141927
4,1bollard.PNG,1365,613,1,1026,266,1065,361,0.054539,0.438151,0.035871,0.141927
...,...,...,...,...,...,...,...,...,...,...,...,...
74,traffic_light38.png,1366,768,8,570,262,610,342,0.054539,0.438151,0.035871,0.141927
75,traffic_light40.png,1366,768,8,249,288,299,398,0.054539,0.438151,0.035871,0.141927
76,traffic_light40.png,1366,768,8,569,292,619,402,0.054539,0.438151,0.035871,0.141927
77,traffic_light40.png,1366,768,8,808,309,858,419,0.054539,0.438151,0.035871,0.141927


In [168]:
bbox

[0.4522058823529412,
 0.799837925445705,
 0.09852941176470588,
 0.2025931928687196]

In [172]:
 a['xx'][0:1]=bbox[0]

C:\Users\VyshakB\.conda\envs\torch\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [173]:
a

,filename,width,height,class,xmin,ymin,xmax,ymax,xx,yy,xy,yx
0,10cabinet.PNG,1360,617,5,548,431,682,556,0.452206,0.438151,0.035871,0.141927
1,11cabinet.PNG,1359,611,5,237,346,432,512,0.054539,0.438151,0.035871,0.141927
2,12cabinet.PNG,1364,616,5,485,302,601,379,0.054539,0.438151,0.035871,0.141927
3,1bollard.PNG,1365,613,1,888,262,916,349,0.054539,0.438151,0.035871,0.141927
4,1bollard.PNG,1365,613,1,1026,266,1065,361,0.054539,0.438151,0.035871,0.141927
...,...,...,...,...,...,...,...,...,...,...,...,...
74,traffic_light38.png,1366,768,8,570,262,610,342,0.054539,0.438151,0.035871,0.141927
75,traffic_light40.png,1366,768,8,249,288,299,398,0.054539,0.438151,0.035871,0.141927
76,traffic_light40.png,1366,768,8,569,292,619,402,0.054539,0.438151,0.035871,0.141927
77,traffic_light40.png,1366,768,8,808,309,858,419,0.054539,0.438151,0.035871,0.141927
